In [1]:
#generic imports

import re
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import seaborn as sn
import matplotlib.pyplot as plt
from functools import partial

#Custom Imports

from CMstyle import *
from functions import *


#custom inlines

%matplotlib inline

#Import Raw Data

df = pd.read_csv('original.csv')  

#import Spell_check data

spell_df = pd.read_csv('spell_check.csv')


/home/aschonn/anaconda3/envs/jupyter/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


I have no idea who I'm picking to go all the way 😉 My @yahoofantasy Tourney Pick’em challenge is…  https://t.co/rTwRiDTOjQ
1.9073486328125e-06


In [2]:
#Randomize Order Of Data

from sklearn.utils import shuffle

df = shuffle(df)
df.reset_index(drop=True, inplace=True)


spell_df = shuffle(spell_df)
spell_df.reset_index(drop=True, inplace=True)

spell_df.head()

,username,tweet
0,money23green,I'm thankful to be placed in a situation where...
1,kdtrey5,T @lovelyone88: @KDthunderup I want rocks mode...
2,ddlovato,Me at The Goo's show!! http://t.co/NgZLbO2q
3,tedcruz,The Kay Bailey Hutchison Convention Center in ...
4,money23green,@SoFarAhead__ lol what u gone do here


In [3]:
df.head()

,username,tweet
0,ddlovato,Or this one.....?!! http://say.ly/WGsnER
1,elonmusk,@CatherineUllo Glass corset hand-painted in pe...
2,money23green,@Tm4_Bam @10mjones lol don't I always beat y'a...
3,elonmusk,"It can transform into a robot, fight aliens an..."
4,money23green,@kuddib lmao u OC bro


In [4]:
#OUR USERS

my_tags = []
for item in df['username']:
    if item not in my_tags:
        my_tags.append(item)
print(my_tags)
        

['ddlovato', 'elonmusk', 'money23green', 'kdtrey5', 'tedcruz']


In [5]:
#Our Features

data_features = df.columns.values
print(data_features)

['username' 'tweet']


In [6]:
#Main Imports For Plots

from pretty_matrix import *
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix



my_tags = ['tedcruz', 'elonmusk', 'kdtrey5', 'ddlovato', 'money23green']


class Plots:
    
    
    def __init__(self, algorithm, functionName):
    
        self.algo = algorithm
        self.functionName = functionName
        
        
        
    def createFolders(self, folder):
        
        if not os.path.isdir(folder):
            os.mkdir(folder)
                        
        if not os.path.isdir(folder + '/' + self.algo):
            os.mkdir(folder + '/' + self.algo)

        
      
    def picOfConfusion(self, pipeline, y_test, y_pred):

        
        folder = "confusionPics"
        filename = folder + '/' + self.algo + '/' + self.functionName + '_CM.png'
        
        self.createFolders(folder)
        
        #import download from 
        # https://github.com/wcipriano/pretty-print-confusion-matrix/blob/master/confusion_matrix_pretty_print.py
        
        plot_confusion_matrix_from_data(y_test, 
                                        y_pred, 
                                        columns = my_tags, 
                                        title = self.algo,
                                        functionName = self.functionName
                                       )

        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.show()
        
        

    def classification_report_plot(self, clf_report, functionName):
                
        folder = "classificationPics"
        filename = folder + '/' + self.algo + '/' + self.functionName + '_CM.png'
        
        self.createFolders(folder)
        
        sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True, cmap="YlGnBu",  fmt='.3g')
        
        plt.title('Classification: ' + self.algo + ' ' + functionName)
        
        plt.savefig(filename, dpi=100)
        
        plt.show()
        
        plt.close()
        
        


In [11]:
#Imports around class

import time
import itertools
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

#RESULTS AFTER COMPLETION LOCATION

BASE = dict()
NB_results = dict()
L_SVC_results = dict()
LR_results = dict()
KNN_results = dict()
DT_results = dict()
RF_results = dict()
xgboost_results = dict()

def preprocess(dataset, testSize, random_state = 0):

    X = df.tweet
    y = df.username

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state=32)

    return X_train, X_test, y_train, y_test


def alterDataByFunction(dataset, function):
    
    dataset['tweet'] = dataset['tweet'].apply(lambda x: function(x))
    
    return dataset


class Algorithms:
      
    def __init__(self, dataset, functionName):
        
        self.functionName = functionName.__name__
        
        if self.functionName == 'spell_check':
            self.altereddataset = spell_df
        else:
            self.altereddataset = dataset

        #Show the data is changing
        
        print(self.functionName)
        print(dataset['tweet'][0:5])
        
        #Split in training and testing sets
        
        X_train, X_test, y_train, y_test = preprocess(self.altereddataset, 0.3, random_state = 32)
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

##############################################################################################################
        
    def NB(self):
        
        #Import Classifier
        
        from sklearn.naive_bayes import MultinomialNB, GaussianNB

        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        for count in range(5):

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultinomialNB()),
                       ])
                        
        # Calculate ROC_AUC
        # There are two options for average: macro or weighted
        # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
        
        #create confusion matrix and classification files (Set for only base)
        
        if self.functionName == 'base' or self.functionName == 'rm_punctuation':

            algorithm = 'Naive Bayes'
            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True),self.functionName)

        #Puts accuracy and speed into dictionary for later
#         accuracy/5, speed/5, percision/5, f1/5, recall/5, roc_score/5
                                
        NB_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]

#####################################################################################################################
        
    def L_SVC(self):
        
        #Import Classifier
        
        from sklearn.svm import LinearSVC
        
        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        #Runs through algorithm 5 times to get an average accuracy
        
        for count in range(5):
            
            print('L_scv', count)

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', LinearSVC()),
                       ])
            
                        
            # Calculate ROC_AUC
            # There are two options for average: macro or weighted
            # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
            
        #create confusion matrix and classification files (Set for only base)
        
        if self.functionName == 'base' or self.functionName == 'rm_punctuation':

            algorithm = 'L_SVC'
            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True),self.functionName)

        #Puts accuracy and speed into dictionary for later
#         accuracy/5, speed/5, percision/5, f1/5, recall/5, roc_score/5
                                
        L_SVC_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]
        
#######################################################################################################################   
    
    def DT(self):
        
        #Import Classifier

        from sklearn.tree import DecisionTreeClassifier
        
        
        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        #Runs through algorithm 5 times to get an average accuracy
        
        for count in range(5):
            
            print('DT', count)

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', DecisionTreeClassifier()),
                       ])
            
                        
            # Calculate ROC_AUC
            # There are two options for average: macro or weighted
            # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
            
        #create confusion matrix and classification files (Set for only base)
        
        if self.functionName == 'base' or self.functionName == 'rm_punctuation':

            algorithm = 'Decision Tree'
            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True),self.functionName)

        #Puts accuracy and speed into dictionary for later
#         accuracy/5, speed/5, percision/5, f1/5, recall/5, roc_score/5
                                
        DT_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]
        
##########################################################################################################################

    def RF(self):
        
        #Import Classifier

        from sklearn.ensemble import RandomForestClassifier
        
        
        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        #Runs through algorithm 5 times to get an average accuracy
        
        for count in range(5):
            
            print('RF', count)

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', RandomForestClassifier()),
                       ])
            
                        
            # Calculate ROC_AUC
            # There are two options for average: macro or weighted
            # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
            
        #create confusion matrix and classification files (Set for only base)
        
        if self.functionName == 'base' or self.functionName == 'rm_punctuation':

            algorithm = 'Random Forest'
            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True),self.functionName)

        #Puts accuracy and speed into dictionary for later
#         accuracy/5, speed/5, percision/5, f1/5, recall/5, roc_score/5
        RF_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]
#################################################################################################################     
        
    def KNN(self):
        
        #Import Classifier

        from sklearn.neighbors import KNeighborsClassifier

        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        #Runs through algorithm 5 times to get an average accuracy
        
        for count in range(5):
            
            print('KNN ', count)

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', KNeighborsClassifier()),
                       ])
            
                        
            # Calculate ROC_AUC
            # There are two options for average: macro or weighted
            # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
            
        #create confusion matrix and classification files (Set for only base)
        
        if self.functionName == 'base':
        
            algorithm = 'K-Nearest Neighbor'

            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True), self.functionName)

        #Puts accuracy and speed into dictionary for later
        
        KNN_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]

###################################################################################################################

    def LR(self):
        
        #Import Classifier
        
        from sklearn.linear_model import LogisticRegression
        
        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        #Runs through algorithm 5 times to get an average accuracy
        
        for count in range(5):
            
            print('LR ', count)

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', LogisticRegression(C=100,random_state=0,max_iter=10000, multi_class='multinomial'))
                       ])
            
                        
            # Calculate ROC_AUC
            # There are two options for average: macro or weighted
            # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
            
        #create confusion matrix and classification files (Using only base for demonstration purposes, but can be used for all if needed)
        
        if self.functionName == 'base':
        
            algorithm = 'Logistic Regression'

            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True), self.functionName)            
        
        #Puts accuracy and speed into dictionary for later
       
        LR_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]


#####################################################################################################################
        
    def xgboost(self):
        
        #import classifier

        from xgboost import XGBClassifier
        
        accuracy = 0 
        speed = 0
        percision = 0
        f1 = 0
        recall = 0
        
        #Runs through algorithm 5 times to get an average accuracy
        
        for count in range(5):
            
            print('XG ', count)

            nb = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', XGBClassifier(objective='multi:softmax', num_class=5)),
                       ])
            
                        
            # Calculate ROC_AUC
            # There are two options for average: macro or weighted
            # We are going with Macro since we have a balanced dataset
            
            nb.fit(self.X_train, self.y_train)
            y_pred = nb.predict(self.X_test)
            accuracy += accuracy_score(y_pred, self.y_test)
            percision += precision_score(self.y_test, y_pred, labels = my_tags, average="macro")
            f1 += f1_score(self.y_test, y_pred, labels = my_tags, average="macro")
            recall += recall_score(self.y_test, y_pred, labels = my_tags, average="macro")
            

        #create confusion matrix and classification files
        
        if self.functionName == 'base':
        
            algorithm = 'XGBoost'

            plots = Plots(algorithm, self.functionName)
            plots.picOfConfusion(nb, self.y_test, y_pred)
            plots.classification_report_plot(classification_report(self.y_test, y_pred, output_dict=True),self.functionName)
        
        #Puts accuracy and speed into dictionary for later
        
        xgboost_results[self.functionName] = [accuracy/5, speed/5, percision/5, f1/5, recall/5]

##########################################################################################################
    
    def runall(self):
        
    #Uncomment To Use:
        
        #Round 1
        
#         self.DT()
#         self.RF()
#         self.L_SVC()
#         self.NB()
        
        #Round 2
        
#         self.xgboost()
#         self.KNN()
        self.LR()
        

In [ ]:
from functions import *
import time
# !pip install autocorrect

functions = [
        base,
        rm_punctuation,
        lemmatize,
        rm_stopwords,
        rm_numbers,
        stemming,
        rp_elongated,
        rm_noise,
        rp_twitterHandles,
        contraction,
        rp_slang,
        spell_check,
]

def tester(df):
    
    for function in functions:
        
        print('Starting: ', function.__name__)
        
        #Sends a copy of our altered dataframe to our class algorithms and runs each algorithm and places the results in the dictionaries above
        Algorithms(alterDataByFunction(df.copy(), function), function).runall()
    
    
tester(df)


Starting:  base
base
0           Or this one.....?!!   http://say.ly/WGsnER
1    @CatherineUllo Glass corset hand-painted in pe...
2    @Tm4_Bam @10mjones lol don't I always beat y'a...
3    It can transform into a robot, fight aliens an...
4                                @kuddib lmao u OC bro
Name: tweet, dtype: object
LR  0


In [ ]:
#Print Out Results

#round 1

print(NB_results)
# print(L_SVC_results)
# print(DT_results)
# print(RF_results)

#round 2

print(LR_results)
# print(KNN_results)
# print(xgboost_results)

In [ ]:
sort_orders = sorted(NB_results.items(), key=lambda x: x[1][1], reverse=True)  
for i in sort_orders:
    print(i[0], i[1]) 

In [ ]:
def techniquePlot(dictionary, name, metric):
    
    if metric.lower() == 'accuracy':
        scoreName = 'Accuracy'
        scoreIndex = 0
        
    #index1 is for speed but it's not ready yet
    if metric.lower() == 'speed':
        scoreName = 'Speed'
        scoreIndex = 1
    
    if metric.lower() == 'precision':
        scoreName = 'Precision'
        scoreIndex = 2
    if metric.lower() == 'f1':
        scoreName = 'F1'
        scoreIndex = 3
    if metric.lower() == 'recall':
        scoreName = 'Recall'
        scoreIndex = 4

    
    
    #folder storage

    folder = 'Technique' + scoreName + 'Plot'
    filename = folder + '/' + name + scoreName + '.png'

    if not os.path.isdir(folder):
        os.mkdir(folder)

    techniques=[]
    accuracy =['Base', scoreName]
    bar_width = .20
    baseline = []
    preformance=[]

    for key, value in dictionary.items():

        techniques.append(key)
        preformance.append(value[scoreIndex] * 100)

    for count in range(len(preformance)):

        baseline.append(preformance[scoreIndex])
    

    pos = np.arange(len(techniques))

    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['axes.axisbelow'] = True
    plt.figure(figsize=(25,10))
    plt.grid(axis = 'y', zorder=0)
    plt.bar(pos, baseline, bar_width,color='blue',edgecolor='black')
    plt.bar(pos + bar_width, preformance, bar_width, color='pink', edgecolor='black')
    plt.xticks(pos, techniques)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=20)
    plt.ylim([min(preformance) - .05, max(preformance) + .05])
    plt.legend(accuracy, fontsize = 20, loc='upper left', frameon=False)
    plt.xlabel('Techniques', fontsize=25)
    plt.ylabel(scoreName + ' (%)', fontsize=25)
    plt.title('Technique ' + scoreName + ' Comparison ' + '- ' + name, fontsize=18)
    plt.savefig(filename, bbox_inches="tight")
    plt.show()

In [ ]:
#Uncomment To Use:

#round 1

# techniqueComparisonPlot(NB_results, 'Naive Bayes')

scores = ['accuracy', 'recall', 'precision', 'f1']

#linear support vector machine

for score in scores:

#     techniquePlot(L_SVC_results, 'Linear Support vector machines', score)
#     techniquePlot(DT_results, 'Decision Tree', score)
#     techniquePlot(RF_results, 'Random Forest', score)

#     techniquePlot(NB_results, 'Naive Bayes', score)

#     techniquePlot(KNN_results, 'KNN', score)
#     techniquePlot(xgboost_results, 'XGBoost', score)
    techniquePlot(LR_results, 'Logistic Regression', score)


# #round 2

# techniqueComparisonPlot(KNN_results, 'KNN')
# techniqueComparisonPlot(xgboost_results, 'XGBoost')
# techniqueComparisonPlot(LR_results, 'Logistic Regression')
